In [1]:
import numpy as np
import pandas as pd

In [19]:
df = pd.read_csv("MF_imputed.csv")

In [20]:
df.head()

,y,Age,SibSp,ParCh,Fare,Duplicate_Cabin,Cab_Letter_B,Cab_Letter_C,Cab_Letter_D,Cab_Letter_E,...,Cab_Letter_G,Cab_Letter_None,Cab_Letter_T,Family_Size,Fare_Per_Person,Sex_male,Embarked__Q,Embarked__S,Class_2,Class_3
0,0.0,22.0,1.0,0.0,7.2,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,3.6,1.0,0.0,1.0,0.0,1.0
1,1.0,38.0,1.0,0.0,71.3,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,35.6,0.0,0.0,0.0,0.0,0.0
2,1.0,26.0,0.0,0.0,7.9,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,7.9,0.0,0.0,1.0,0.0,1.0
3,1.0,35.0,1.0,0.0,53.1,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,26.6,0.0,0.0,1.0,0.0,0.0
4,0.0,35.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,8.0,1.0,0.0,1.0,0.0,1.0


In [21]:
df = df.astype({'y': 'int64'}).copy()

In [24]:
df.head()

,y,Age,SibSp,ParCh,Fare,Duplicate_Cabin,Cab_Letter_B,Cab_Letter_C,Cab_Letter_D,Cab_Letter_E,...,Cab_Letter_G,Cab_Letter_None,Cab_Letter_T,Family_Size,Fare_Per_Person,Sex_male,Embarked__Q,Embarked__S,Class_2,Class_3
0,0,22.0,1.0,0.0,7.2,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,3.6,1.0,0.0,1.0,0.0,1.0
1,1,38.0,1.0,0.0,71.3,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,35.6,0.0,0.0,0.0,0.0,0.0
2,1,26.0,0.0,0.0,7.9,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,7.9,0.0,0.0,1.0,0.0,1.0
3,1,35.0,1.0,0.0,53.1,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,26.6,0.0,0.0,1.0,0.0,0.0
4,0,35.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,8.0,1.0,0.0,1.0,0.0,1.0


# Isolate X and Y 

In [25]:
y = df.y
X = df.drop(columns='y',axis=1)
X.head()

,Age,SibSp,ParCh,Fare,Duplicate_Cabin,Cab_Letter_B,Cab_Letter_C,Cab_Letter_D,Cab_Letter_E,Cab_Letter_F,Cab_Letter_G,Cab_Letter_None,Cab_Letter_T,Family_Size,Fare_Per_Person,Sex_male,Embarked__Q,Embarked__S,Class_2,Class_3
0,22.0,1.0,0.0,7.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,3.6,1.0,0.0,1.0,0.0,1.0
1,38.0,1.0,0.0,71.3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,35.6,0.0,0.0,0.0,0.0,0.0
2,26.0,0.0,0.0,7.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,7.9,0.0,0.0,1.0,0.0,1.0
3,35.0,1.0,0.0,53.1,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,26.6,0.0,0.0,1.0,0.0,0.0
4,35.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,8.0,1.0,0.0,1.0,0.0,1.0


# Train, Test, Split

In [26]:
from sklearn.model_selection import train_test_split

In [27]:
df.y.mean()

0.3838383838383838

Data is imblanaced, 38% survived so we will use stratify to ensure balanced split

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.2,
                                                    random_state = 1502,
                                                    stratify = y)

# RandomForestClassifier

In [29]:
from sklearn.ensemble import RandomForestClassifier

In [30]:
model = RandomForestClassifier(n_estimators = 300,
                               random_state = 1502)
model.fit(X_train, y_train)

RandomForestClassifier(n_estimators=300, random_state=1502)

# Predictions

In [31]:
predictions = model.predict(X_test)
predictions[:5]

array([1, 0, 1, 0, 1], dtype=int64)

# Assessing the model

In [32]:
from sklearn.metrics import classification_report, f1_score

In [33]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.83      0.85      0.84       110
           1       0.76      0.72      0.74        69

    accuracy                           0.80       179
   macro avg       0.79      0.79      0.79       179
weighted avg       0.80      0.80      0.80       179



In [34]:
f1_score(y_test, predictions)

0.7407407407407408

# Feature importance

In [35]:
importance = pd.Series(model.feature_importances_,
                       index = X_train.columns.values)

importance.sort_values(ascending=False)

Age                0.268583
Sex_male           0.216740
Fare_Per_Person    0.135852
Fare               0.126798
Class_3            0.041614
Family_Size        0.036431
Cab_Letter_None    0.032338
SibSp              0.025512
Embarked__S        0.021135
ParCh              0.020438
Duplicate_Cabin    0.017284
Class_2            0.013371
Cab_Letter_E       0.010719
Embarked__Q        0.010560
Cab_Letter_D       0.007267
Cab_Letter_C       0.007007
Cab_Letter_B       0.004436
Cab_Letter_F       0.002329
Cab_Letter_G       0.001090
Cab_Letter_T       0.000498
dtype: float64

# Improving the model with parameter tuning

We're tuning the number of estimators and number of trees

In [36]:
from sklearn.model_selection import ParameterGrid

In [51]:
param_grid = {'n_estimators': range(200,250,5)}
param_grid

{'n_estimators': range(200, 250, 5)}

In [52]:
grid = ParameterGrid(param_grid)
grid

In [53]:
#Storing results
f1score = []
number = []
i = 0
for p in grid:
  model = RandomForestClassifier(n_estimators = p['n_estimators'],
                                 random_state = 1502)
  model.fit(X_train, y_train)
  predictions = model.predict(X_test)
  f1 = f1_score(y_test, predictions)
  f1score.append(f1)
  number.append(grid[i])
  i += 1

In [54]:
results = pd.DataFrame((zip(number,f1score)),columns=['n_estimators', 'f1score'])

In [55]:
results.sort_values('f1score', ascending=False)

,n_estimators,f1score
1,{'n_estimators': 205},0.746269
2,{'n_estimators': 210},0.746269
3,{'n_estimators': 215},0.746269
4,{'n_estimators': 220},0.746269
5,{'n_estimators': 225},0.746269
6,{'n_estimators': 230},0.746269
7,{'n_estimators': 235},0.746269
8,{'n_estimators': 240},0.746269
0,{'n_estimators': 200},0.740741
9,{'n_estimators': 245},0.740741


In [45]:
print(max(f1score))

0.7462686567164178


#  Submission

## Trained model

In [56]:
train = df.copy()
train_y = train.y

In [57]:
train.columns

Index(['y', 'Age', 'SibSp', 'ParCh', 'Fare', 'Duplicate_Cabin', 'Cab_Letter_B',
       'Cab_Letter_C', 'Cab_Letter_D', 'Cab_Letter_E', 'Cab_Letter_F',
       'Cab_Letter_G', 'Cab_Letter_None', 'Cab_Letter_T', 'Family_Size',
       'Fare_Per_Person', 'Sex_male', 'Embarked__Q', 'Embarked__S', 'Class_2',
       'Class_3'],
      dtype='object')

In [59]:
predictors_cols = ['Age', 'SibSp', 'ParCh', 'Fare', 'Duplicate_Cabin', 'Cab_Letter_B',
       'Cab_Letter_C', 'Cab_Letter_D', 'Cab_Letter_E', 'Cab_Letter_F',
       'Cab_Letter_G', 'Cab_Letter_None', 'Cab_Letter_T', 'Family_Size',
       'Fare_Per_Person', 'Sex_male', 'Embarked__Q', 'Embarked__S', 'Class_2',
       'Class_3']

In [60]:
train_X = train[predictors_cols]

In [61]:
model = RandomForestClassifier(n_estimators = 205,
                               random_state = 1502)
model.fit(train_X, train_y)

RandomForestClassifier(n_estimators=205, random_state=1502)

## Test data

In [ ]:
test = pd.read_csv('test_MF_imputed.csv')

In [ ]:
test.shape

In [ ]:
test.head()

In [ ]:
train_X.head(0), test.head(0)

In [ ]:
test_X = test[predictors_cols]

In [ ]:
predicted = model.predict(test_X)

In [ ]:
test['Survived'] = predicted

In [ ]:
test.head()

In [ ]:
test = test[['PassengerId', 
                    #'Age', 'SibSp', 
                    #'ParCh', 'Fare', 
                    #'Class_2', 'Class_3',
                   #'Sex_male', 
                    'Survived']].copy()

In [ ]:
test.dtypes

In [ ]:
test = test.astype({'PassengerId': 'int64'}).copy()

In [ ]:
test.dtypes

In [ ]:
test.to_csv('RandomForestClassifier_Submission.csv', index=False)